In [19]:
import re
import requests
import html2text
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq
import codecs

## 从首页提取所有链接

In [2]:
url = 'https://www.w3.org/DesignIssues/'

In [3]:
r = requests.get(url)

In [4]:
html = r.text

In [5]:
soup = BeautifulSoup(html, 'html.parser')

### 这里遇到了一点小坑, 就是直接上首页无法用 pyquery 解析到 a 标签
- 所以先提取 body 部分载进一步提取

In [6]:
body = soup.body

In [7]:
d = pq(str(body))

In [8]:
li = d('li')

In [9]:
a = li('a')

In [10]:
urls_dict = {}

In [11]:
for a in a.items():
    href = str(a.attr('href'))
    urls_dict[href] = url+href

### 去掉一些无用链接

In [12]:
urls_dict.pop('../1999/09/specification.html')

'https://www.w3.org/DesignIssues/../1999/09/specification.html'

In [13]:
urls_dict.pop('../Protocols/RelevantProtocols.html')

'https://www.w3.org/DesignIssues/../Protocols/RelevantProtocols.html'

In [14]:
urls_dict.pop('../MarkUp/SGML/TimComments.html')

'https://www.w3.org/DesignIssues/../MarkUp/SGML/TimComments.html'

In [15]:
urls_dict.pop('http://www.w3.org/2003/04/iri.html')

'https://www.w3.org/DesignIssues/http://www.w3.org/2003/04/iri.html'

In [16]:
urls_dict

{'Abstractions.html': 'https://www.w3.org/DesignIssues/Abstractions.html',
 'Annotation': 'https://www.w3.org/DesignIssues/Annotation',
 'Architecture.html': 'https://www.w3.org/DesignIssues/Architecture.html',
 'Availability.html': 'https://www.w3.org/DesignIssues/Availability.html',
 'Axioms.html': 'https://www.w3.org/DesignIssues/Axioms.html',
 'BuildingBackLinks': 'https://www.w3.org/DesignIssues/BuildingBackLinks',
 'CG.html': 'https://www.w3.org/DesignIssues/CG.html',
 'Caching.html': 'https://www.w3.org/DesignIssues/Caching.html',
 'CloudStorage.html': 'https://www.w3.org/DesignIssues/CloudStorage.html',
 'CompatibleProof': 'https://www.w3.org/DesignIssues/CompatibleProof',
 'Conversations': 'https://www.w3.org/DesignIssues/Conversations',
 'Diff': 'https://www.w3.org/DesignIssues/Diff',
 'Editor.html': 'https://www.w3.org/DesignIssues/Editor.html',
 'Evolution.html': 'https://www.w3.org/DesignIssues/Evolution.html',
 'Extensible.html': 'https://www.w3.org/DesignIssues/Extensibl

In [17]:
len(urls_dict)

82

## 提取相应链接的文档
- 需要解决一个问题: 文章中的 local 链接如何替换成 global url
- 根据观察, 有以下几个模式
    - \[](sjflskdjf.html)
    - \[](sdjf-sjd.html)
    - \[](skjdf0.9sdfs.html)
    - \[](../sfsdfjlk.html)
    - \[](../../sdfkj.html)
    - \[](sdfsdf.html#4)

- 其实不用管后边, 只要处理好 `](`, `](../../` 这种应该就行了.
- 还要求不能匹配 `](http` 这种模式

In [163]:
def sub_url(text):
    p_1 = re.compile(r'(\]\()((?!http)(?!\.\.))', re.VERBOSE | re.MULTILINE)
    p_2 = re.compile(r'(\]\(\.\.\/)((?!\.\.))', re.VERBOSE | re.MULTILINE)
    sub_part_1 = '](https://www.w3.org/DesignIssues/'
    sub_part_2 = '](https://www.w3.org/'
    text1 = p_1.sub(sub_part_1, text)
    text2 = p_2.sub(sub_part_2, text1)
    # result = p.findall(text)
    return text2

with codecs.open('Semantic.html.md', 'r') as f:
    text = f.read()
    
text = sub_url(text)
# print text

In [164]:
i = 0
for k, v in urls_dict.items():
    print str(i) + '-->' + v
    r = requests.get(v)
    html = r.text
    original_text = html2text.html2text(html)
    filtered_text = sub_url(original_text)
    with open('%s.md' % k, 'w') as f:       
        f.write(filtered_text.encode('utf-8'))
    i += 1

0-->https://www.w3.org/DesignIssues/Semantic.html
1-->https://www.w3.org/DesignIssues/LinkMyths.html
2-->https://www.w3.org/DesignIssues/Security-ClientCerts.html
3-->https://www.w3.org/DesignIssues/CompatibleProof
4-->https://www.w3.org/DesignIssues/ProtocolVersions.html
5-->https://www.w3.org/DesignIssues/Multiuser.html
6-->https://www.w3.org/DesignIssues/Caching.html
7-->https://www.w3.org/DesignIssues/Conversations
8-->https://www.w3.org/DesignIssues/Model.html
9-->https://www.w3.org/DesignIssues/NamespacesAreResources.html
10-->https://www.w3.org/DesignIssues/Annotation
11-->https://www.w3.org/DesignIssues/CG.html
12-->https://www.w3.org/DesignIssues/Meaning.html
13-->https://www.w3.org/DesignIssues/PersistentDomains.html
14-->https://www.w3.org/DesignIssues/RDF-XML.html
15-->https://www.w3.org/DesignIssues/Navigation.html
16-->https://www.w3.org/DesignIssues/SemanticClipboard
17-->https://www.w3.org/DesignIssues/RDB-RDF.html
18-->https://www.w3.org/DesignIssues/Syntax.html
19-->h

## The whole page

- 先想的是整个转化成 Markdown

In [32]:
soup_str = str(soup)

In [35]:
summary = str(html2text.html2text(soup_str))

In [41]:
p = re.compile(r'\.html\)')

In [42]:
summary = p.sub(r'.html.md)', summary)

In [44]:
with open('SUMMARY.md', 'w') as f:
    f.write(summary)